# Regressão Logística Simples

### Descrição:
Nesse notebook iremos desenvolver e executar um algoritmo de Regressão Logística. Essa técnica é uma das mais simples técnicas de Classificação, e é extremamente similar à Regressão Linear. Utilizaremos uma versão simplificada do famoso Iris Dataset (Classificação de Flores).

A Regressão Logística irá estimar parâmetros para um Modelo de Classificação. Isto é, o nosso modelo não irá prever um valor contínuo, como era na Regressão Linear. Ao invés disso, as predições irão informar uma probabilidade de tal exemplar pertencer (ou não) a determinada classe já estabelecida. O treinamento, então, busca determinar a chamada "Linha de Decisão", que separa linearmente quais exemplares pertencem a uma classe e quais não pertencem.

<b> Obs.: </b> todas as matrizes/vetores utilizados na fundamentação teórica são consideradas como Vetores-Colunas. A implementação pode diferir um pouco dessa convenção.


## Bibliotecas e Configurações

In [2]:
# -*- coding: utf-8 -*-
%matplotlib qt5

# Libraries
import numpy as np 
import matplotlib.pyplot as plt
import random as rnd

## Modelo Logístico

Na célula abaixo, iremos programar nosso Modelo Logístico. Diferentemente da Regressão Linear, o nosso modelo depende de uma função não-linear (a própria função logística). Trata-se, justamente, da Sigmoide, definida da seguinte forma:

$$ g(X) = \frac{1}{1+e^{-x}} $$

Dessa forma, o nosso modelo final será

$$ h(\theta) = g(\theta^{T}X) = \frac{1}{1+e^{-\theta^{T}X}} $$

In [3]:
# Definição das Funções Sigmoide e Modelo Logístico
def sigmoid(X):
    ''' Returns the Sigmoid Function applied to a vector (or value) x '''
    return 1 / (1 + np.exp(-1 * X))
    
def h_theta(X, theta):
    ''' Apply the Linear Model for features X and parameters theta '''
    return sigmoid(np.dot(np.transpose(theta), X))

# Teste da Sigmoide
testSigmoid = np.linspace(-30, 30)

plt.figure(1)
plt.title("Sigmoid Function")
plt.plot(testSigmoid, sigmoid(testSigmoid), 'b-')
plt.show()

## Função de Acurácia

Em problemas de Classificação, subtrair o valor previsto do valor real não nos dá uma informação muito precisa (sobre, por exemplo, a gravidade do erro). Para avaliar melhor a qualidade de um Treinamento de Classificação existem diversas medidas. Uma bem comum, e simples, consiste na acurácia: a quantidade de exemplos "corretamente classificados". Vamos definir essa porcentagem como sendo:

$$ Acc(\theta) = 100 \times (1 - \frac{1}{m} \sum (h(\theta) - y)^{2}) $$

<b>Obs.:</b> a acurácia só será correta caso tanto $h(\theta)$ quanto $y$ sejam valores binários. Logo, iremos arredondar os resultados de $h(\theta)$. Os valores de $y$ já são binários.

In [4]:
def accuracyFunction(X, y, theta):
    ''' Calculates the percentage of correct classifications '''
    Y_pred = h_theta(X, theta)
    pos = np.where(Y_pred >= 0.5); Y_pred[pos] = 1
    neg = np.where(Y_pred < 0.5); Y_pred[neg] = 0
    
    return 100 * (1 - (1 / np.size(y)) * np.sum((Y_pred - y) ** 2))

## Linha de Decisão

Na célula abaixo iremos definir uma função simples para calcular e retornar os eixos cartesianos da Linha de Decisão estimada pelo modelo. Essa função apenas utiliza os parâmetros $\theta$ para calcular o coeficiente angular e linear da reta.

In [5]:
def decisionBound(theta, x):
    ''' Calculates and returns a linear Decision Boundary from the model '''
    boundary_X = np.linspace(min(x[1,:]), max(x[1,:]))
    boundary_Y = -1*(theta[1] / theta[2]) * boundary_X - (theta[0] / theta[2]);
    return [boundary_X, boundary_Y]

## Programa Principal (Regressão Logística)

No programa principal, iremos programar a Regressão Logística propriamente dita.
O treinamento é <b>exatamente o mesmo</b> da Regressão Linear.
Dividimos o código em três partes:

### Part 1: Data Pre-Processing

Nesse trecho, iremos nos preocupar em carregar e organizar o dataset que utilizaremos no treinamento. É nesse momento, também, que iremos declarar e separar as variáveis que definem o Conjunto de Atributos, o Conjunto de Saída e os Parâmetros do Modelo, além dos Hiperparâmetros de Treinamento. Iremos seguir a convenção de considerar todos os exemplares como vetores-colunas. No entanto, o numpy não nos permite facilmente modificar essa informação para o Conjunto de Saída, e o mesmo continuará como vetor-linha (sem muito prejuízo). Iremos criar, também um vetor para armazenar o Histórico de Erros do treinamento (por motivos de visualização).

Iremos utilizar o dataset <i>irisDataSimple.txt</i> localizado na pasta <i>datasets/</i>. Teremos as seguintes matrizes:

$$
    X = \begin{bmatrix} 1 & 1 & \cdots & 1 \\  X_{1}^{(1)} & X_{1}^{(2)} & \cdots & X_{1}^{(m)} \\  X_{2}^{(1)} & X_{2}^{(2)} & \cdots & X_{2}^{(m)} \end{bmatrix};\   \theta = \begin{bmatrix} \theta_{0}\\ \theta_{1} \\ \theta_{2}\end{bmatrix};\  Y = \begin{bmatrix} Y^{(1)} & Y^{(2)} & \cdots & Y^{(m)} \end{bmatrix}
$$

### Part 2: Linear Regression Training

Para cada época até a convergência (ou até atingir o limite máximo definido pelo Hiperparâmetro) iremos realizar o Treinamento da Regressão Linear. Os passos serão os seguintes:

1. Calculamos o vetor de predição "Y_pred", como resultado da predição do Modelo para os parâmetros daquela época;
2. Utilizando "Y_pred", calculamos os erros de acordo com o a matriz real "Y";
3. Concatenamos o Custo Total do erro calculado no Histórico de Erros;
4. Realizamos, para cada parâmetro, o Gradiente Descendente para estimar os novos valores dos parâmetros;
5. Imprimimos os resultados do treino a cada 10 épocas;
6. Verificamos uma possível convergência do treino, e paremos o mesmo caso seja verificado;

### Part 3: Data Plotting and Training Results

Ao fim do treinamento, iremos plotar duas figuras para avaliar o resultado final do nosso algoritmo. A <b>Figura 1</b> irá apenas exibir os atributos do Dataset. A <b>Figura 2</b> irá exibir a Linha de Decisão, além do Histórico de Erros dado as épocas até convergência.

In [21]:
#  Main Function
if __name__=='__main__':
   
    ###############################
    # Part 1: Data Pre-Processing #
    ###############################
    # Loads the data
    data = np.loadtxt("../datasets/irisDataSimple.txt")
    
    n_examples = np.size(data,0)
    n_features = np.size(data,1)
    
    # Define the model parameters
    x = np.array([np.ones(n_examples), data[:, 0], data[:, 1]])
    y = data[:, -1]
    theta = np.zeros([np.size(x, 0), 1])
    
    # Defines the hyperparameters and training measurements
    alfa = 0.5
    max_epochs = 50000
    
    error_hist = np.zeros([max_epochs])
    epsilon = 0.01
    
    ######################################
    # Part 2: Linear Regression Training #
    ######################################
    for epochs in range(max_epochs):
        # Randomly shuffle the data
        randomIndex = rnd.sample(range(n_examples), n_examples)
        x = x[:, randomIndex]
        y = y[randomIndex]
        
        # Calculate the error vector from the current Model
        y_pred = h_theta(x, theta)
        error = y_pred - y
        
        # Append new Least Square Error to History
        error_hist[epochs] = accuracyFunction(x, y, theta)

        # Perform Stochastic Gradient Descent
        for j in range(n_features):
            for k in range(n_examples):
                theta[j] = theta[j] - (alfa/n_examples) * (error[0,k] * x[j,k])

        # Prints training status at each 50 epochs    
        if(epochs % 10 == 0):
            print("###### Epoch", epochs, "######")
            print("Error:", error_hist[epochs], "%")
            print("Thetas:\n", theta)
            print("")
        
        # Evaluate convergence and stops training if so
        if(abs(error_hist[epochs] - error_hist[epochs-50]) <= epsilon):
            print("Gradient Converged!!!\nStopping at epoch", epochs)
            print("###### Epoch", epochs, "######")
            print("Error:", error_hist[epochs], "%")
            print("Thetas:\n", theta)
            break
    
    #############################################
    # Part 3: Data Plotting and Training Result #
    #############################################
    # First Figure: Dataset plotting
    plt.figure(2)
    
    plt.title("Simplified Iris Dataset Classification\n(Green=Iris-setosa; Blue=Iris-virginica)")
    plt.xlabel("Sepal width (cm)")
    plt.ylabel("Sepal length (cm)")
    
    pos = np.where(y == 1)
    neg = np.where(y == 0)
    
    plt.grid()
    plt.plot(x[1,pos], x[2,pos], 'go', x[1,neg], x[2,neg], 'bo')
    
    plt.show()

    # Second Figure: Training results
    plt.figure(3)
    deciBound = decisionBound(theta, x)
    
    plt.subplot(1,2,1)
    
    plt.title("Decision Boundary\n(Green=Iris-setosa; Blue=Iris-virginica; Black=DecisionBoundary)")
    plt.xlabel("Sepal width (cm)")
    plt.ylabel("Sepal length (cm)")
    
    plt.grid()
    plt.plot(x[1,pos], x[2,pos], 'go', x[1,neg], x[2,neg], 'bo', deciBound[0], deciBound[1], 'k-')
    
    plt.subplot(1,2,2)
    
    plt.title("Error History")
    plt.xlabel("Epochs")
    plt.ylabel("Least Square Error")
    
    plt.grid()
    plt.plot(error_hist[:epochs], "g-")
    
    plt.show()

#__

ValueError: could not broadcast input array from shape (100) into shape (1)